In [1]:
import pandas as pd
import numpy as np
import csv
from pathlib import Path

In [1]:
!pwd

/home/johnny/Documents/job-stuff/ElectricTwin/AUT-experiment1/LLM_AUT_test


In [2]:
def read_and_process_and_export_human_AUT(input_file: str, output_file: str, dataset_directory: str, score: float=2., column: str='Responses') -> None:
    data = Path(dataset_directory) / input_file

    if not data.exists():
        print(f"cannot find {data}")

    df = pd.read_csv(data, sep='\t')
    df = df.fillna(0)
    # filter out lower-valued data - this has been evaluated by human scoreres
    df = df[df['average'] >= score]
    # we've got rid of some data, so let's reset the index so we have a sensible ID
    df = df.reset_index(drop=True)
    # export this bad boy

    output_dataset = Path(dataset_directory) / output_file
    
    df.to_csv(output_dataset, columns=[column], sep='\t')
    

In [3]:
dataset_directory = "/home/johnny/Documents/job-stuff/ElectricTwin/AUT-experiment1/LLM_AUT_test/human-responses"

In [4]:
paperclip_input_fnam = "paperclip_AUT_dataset.csv"
paperclip_output_fnam = "paperclip_human_AUT_dataset.tsv"

read_and_process_and_export_human_AUT(input_file = paperclip_input_fnam, 
                                      output_file = paperclip_output_fnam, 
                                      dataset_directory=dataset_directory)

In [5]:
bowl_input_fnam = "bowl_AUT_dataset.csv"
bowl_output_fnam = "bowl_human_AUT_dataset.tsv"

read_and_process_and_export_human_AUT(input_file = bowl_input_fnam, 
                                      output_file = bowl_output_fnam, 
                                      dataset_directory=dataset_directory)

Now we need to create some input files for our evaluation LLM.

We sample the human responses & create 

In [6]:
d = Path(dataset_directory)
paperclip_input_fnam = d / paperclip_output_fnam

In [7]:
paperclip_input_fnam

PosixPath('/home/johnny/Documents/job-stuff/ElectricTwin/AUT-experiment1/LLM_AUT_test/human-responses/paperclip_human_AUT_dataset.tsv')

In [8]:
paperclip_df = pd.read_csv(paperclip_input_fnam, sep='\t')

In [19]:
#paperclip_df

In [20]:
bowl_input_fnam = d / bowl_output_fnam

In [21]:
bowl_input_fnam

PosixPath('/home/johnny/Documents/job-stuff/ElectricTwin/AUT-experiment1/LLM_AUT_test/human-responses/bowl_human_AUT_dataset.tsv')

In [22]:
bowl_df = pd.read_csv(bowl_input_fnam, sep='\t')

In [23]:
bowl_df.head()

,Unnamed: 0,Responses
0,0,put phone inside it to amplify the sound
1,1,trap spider
2,2,giving a haricut
3,3,wear it as a hat
4,4,haircut template


In [10]:
sample_sz = 40
sample_df = paperclip_df.sample(n=sample_sz, random_state=21)

In [11]:
sample_df.head()

,Unnamed: 0,Responses
326,326,as a makeshift filament
265,265,to make bubbles
72,72,clean shallow drain
47,47,use it as an electrical conductor
288,288,heat it to feel how hot metal feels like


In [12]:
def create_samples(df, n=40, random_state=21):
    """Sample from the dataframe until we run out of rows."""
    sample_df = df.sample(n=sample_sz, random_state=random_state)
    df = df.loc[~df.index.isin(sample_df.index)]
    return df, sample_df 

In [13]:
rest_of_the_data_df, thingfish_df = create_samples(paperclip_df)

In [14]:
thingfish_df.count()

Unnamed: 0    40
Responses     40
dtype: int64

In [15]:
rest_of_the_data_df.shape[0]

374

In [16]:
# code's a bit messy, but it's not going to be used more than a couple of times
import csv

def output_to_csv(df, file, field='Responses'):
    fields = df[field].to_list()
    with open(file, 'w') as tsvfile:
        for r in fields:
            tsvfile.write(f"{r}\n")

In [17]:
i = 1
paperclip_copy_df = paperclip_df.copy()

while paperclip_copy_df.shape[0] >= sample_sz:
    paperclip_copy_df, sample_df = create_samples(paperclip_copy_df)
    human_paperclip_input_fnam = d / f"paperclip_human_{i}.tsv"
    #sample_df.to_csv(human_paperclip_input_fnam, sep='\t')
    output_to_csv(df=sample_df, file=human_paperclip_input_fnam, field='Responses')
    print(f"Rows remaining: {paperclip_copy_df.shape[0]}")
    i = i + 1

#paperclip_copy_df.to_csv(human_paperclip_input_fnam, sep='\t')
human_paperclip_input_fnam = d / f"paperclip_human_{i}.tsv"
output_to_csv(df=paperclip_copy_df, file=human_paperclip_input_fnam, field='Responses')

Rows remaining: 374
Rows remaining: 334
Rows remaining: 294
Rows remaining: 254
Rows remaining: 214
Rows remaining: 174
Rows remaining: 134
Rows remaining: 94
Rows remaining: 54
Rows remaining: 14


In [24]:
i = 1
bowl_copy_df = bowl_df.copy()

while bowl_copy_df.shape[0] >= sample_sz:
    bowl_copy_df, sample_df = create_samples(bowl_copy_df)
    human_bowl_input_fnam = d / f"bowl_human_{i}.tsv"
    #sample_df.to_csv(human_bowl_input_fnam, sep='\t')
    output_to_csv(df=sample_df, file=human_bowl_input_fnam, field='Responses')
    print(f"Rows remaining: {bowl_copy_df.shape[0]}")
    i = i + 1

#bowl_copy_df.to_csv(human_bowl_input_fnam, sep='\t')
human_bowl_input_fnam = d / f"bowl_human_{i}.tsv"
output_to_csv(df=bowl_copy_df, file=human_bowl_input_fnam, field='Responses')

Rows remaining: 511
Rows remaining: 471
Rows remaining: 431
Rows remaining: 391
Rows remaining: 351
Rows remaining: 311
Rows remaining: 271
Rows remaining: 231
Rows remaining: 191
Rows remaining: 151
Rows remaining: 111
Rows remaining: 71
Rows remaining: 31
